# Data Analysis of Cardiovascular Study Dataset: Logistic Regression
# An Example of Data Cleaning

This notebook, titled 'cardiovascular_project', presents an analysis of Cardiovascular Study dataset. It includes data loading and cleaning, exploratory data analysis, preprocessing, feature selection, model training, evaluation, and insights. The dataset used is sourced from [Kaggle](https://www.kaggle.com/datasets/christofel04/cardiovascular-study-dataset-predict-heart-disea). The dataset is from an ongoing cardiovascular study on residents of the town of Framingham, Massachusetts. The dataset provides the patients’ information. It includes 3390 records and 15 attributes.

**The project goal** is to build a model that predicts whether the patient has 10-year risk of future coronary heart disease (CHD).

To achieve the goal, logistic regression in Python using the scikit-learn library is used in this project.

**Introduction**

World Health Organization has estimated 12 million deaths occur worldwide, every year due to Heart diseases. Half the deaths in the United States and other developed countries are due to cardio vascular diseases. The early prognosis of cardiovascular diseases can aid in making decisions on lifestyle changes in high risk patients and in turn reduce the complications. This research intends to pinpoint the most relevant/risk factors of heart disease as well as predict the overall risk using logistic regression

**Detailed data description**:

Each attribute is a potential risk factor. There are both demographic, behavioral and medical risk factors.

*Demographic*

• Sex: male or female ('M' or 'F')

• Age: Age of the patient (Continuous - Although the recorded ages have been truncated to whole numbers, the concept of age is continuous.)


*Behavioral*

• is_smoking: whether or not the patient is a current smoker ('YES' or 'NO')

• Cigs Per Day: the number of cigarettes that the person smoked on average in one day (Can be considered continuous as one can have any number of cigarettes, even half a cigarette.)


*Medical(history)*

• BP Meds: whether or not the patient was on blood pressure medication (Nominal)

• Prevalent Stroke: whether or not the patient had previously had a stroke (Nominal)

• Prevalent Hyp: whether or not the patient was hypertensive (Nominal)

• Diabetes: whether or not the patient had diabetes (Nominal)


*Medical(current)*

• Tot Chol: total cholesterol level (Continuous)

• Sys BP: systolic blood pressure (Continuous)

• Dia BP: diastolic blood pressure (Continuous)

• BMI: Body Mass Index (Continuous)

• Heart Rate: heart rate (Continuous - in medical research, variables such as heart rate though in fact discrete, yet are considered continuous because of large number of possible values.)

• Glucose: glucose level (Continuous)


*Predict variable (desired target)*

• 10 year risk of coronary heart disease CHD (binary: '1', means 'YES', '0' means 'NO')

## Data Loading and Cleaning

Importing required libraries and loading the dataset:

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
import cufflinks as cf
import kagglehub
from typing import List
from typing import Tuple
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    StratifiedKFold,
)
from sklearn.metrics import (
    roc_auc_score,
    roc_curve,
    auc,
    classification_report,
    confusion_matrix
)

%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
cf.go_offline();

In [73]:
df_filepath = kagglehub.dataset_download(
    "christofel04/cardiovascular-study-dataset-predict-heart-disea"
)

print("Path to dataset directory:", df_filepath)

files_in_directory = os.listdir(df_filepath)
print("Files in dataset directory:", files_in_directory)

csv_file_path = os.path.join(df_filepath, "train.csv")

chd_data = pd.read_csv(csv_file_path, index_col=0)
chd_data.head()

Path to dataset directory: /home/ubuntu/.cache/kagglehub/datasets/christofel04/cardiovascular-study-dataset-predict-heart-disea/versions/1
Files in dataset directory: ['train.csv', 'test.csv']


,age,education,sex,is_smoking,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
id,,,,,,,,,,,,,,,,
0,64,2.0,F,YES,3.0,0.0,0,0,0,221.0,148.0,85.0,NaN,90.0,80.0,1
1,36,4.0,M,NO,0.0,0.0,0,1,0,212.0,168.0,98.0,29.77,72.0,75.0,0
2,46,1.0,F,YES,10.0,0.0,0,0,0,250.0,116.0,71.0,20.35,88.0,94.0,0
3,50,1.0,M,YES,20.0,0.0,0,1,0,233.0,158.0,88.0,28.26,68.0,94.0,1
4,64,1.0,F,YES,30.0,0.0,0,0,0,241.0,136.5,85.0,26.42,70.0,77.0,0


In [74]:
chd_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3390 entries, 0 to 3389
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   age              3390 non-null   int64  
 1   education        3303 non-null   float64
 2   sex              3390 non-null   object 
 3   is_smoking       3390 non-null   object 
 4   cigsPerDay       3368 non-null   float64
 5   BPMeds           3346 non-null   float64
 6   prevalentStroke  3390 non-null   int64  
 7   prevalentHyp     3390 non-null   int64  
 8   diabetes         3390 non-null   int64  
 9   totChol          3352 non-null   float64
 10  sysBP            3390 non-null   float64
 11  diaBP            3390 non-null   float64
 12  BMI              3376 non-null   float64
 13  heartRate        3389 non-null   float64
 14  glucose          3086 non-null   float64
 15  TenYearCHD       3390 non-null   int64  
dtypes: float64(9), int64(5), object(2)
memory usage: 450.2+ KB


This dataset contains **3390 rows and 16 columns**. The datatype of each column is listed. 

However, the 'sex' and 'is_smoking' columns are of the object data type, but they are categorical by nature. For logistic regression, **categorical values need to be converted to numeric. In the 'sex' column, values are converted to 0 ('F' for female) and 1 ('M' for male). In the 'is_smoking' column, values are converted to 0 ('NO') and 1 ('YES').**

In [75]:
chd_data["sex"] = chd_data["sex"].map({"F": 0, "M": 1}).astype("Int64")
chd_data["is_smoking"] = chd_data["is_smoking"].map({"NO": 0, "YES": 1}).astype("Int64")

print(chd_data.dtypes)

age                  int64
education          float64
sex                  Int64
is_smoking           Int64
cigsPerDay         float64
BPMeds             float64
prevalentStroke      int64
prevalentHyp         int64
diabetes             int64
totChol            float64
sysBP              float64
diaBP              float64
BMI                float64
heartRate          float64
glucose            float64
TenYearCHD           int64
dtype: object


In [76]:
chd_data.describe()

,age,education,sex,is_smoking,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
count,3390.000000,3303.000000,3390.0,3390.0,3368.000000,3346.000000,3390.000000,3390.000000,3390.000000,3352.000000,3390.00000,3390.000000,3376.000000,3389.000000,3086.000000,3390.000000
mean,49.542183,1.970936,0.432743,0.49764,9.069477,0.029886,0.006490,0.315339,0.025664,237.074284,132.60118,82.883038,25.794964,75.977279,82.086520,0.150737
std,8.592878,1.019081,0.495529,0.500068,11.879078,0.170299,0.080309,0.464719,0.158153,45.247430,22.29203,12.023581,4.115449,11.971868,24.244753,0.357846
min,32.000000,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,107.000000,83.50000,48.000000,15.960000,45.000000,40.000000,0.000000
25%,42.000000,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,206.000000,117.00000,74.500000,23.020000,68.000000,71.000000,0.000000
50%,49.000000,2.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,234.000000,128.50000,82.000000,25.380000,75.000000,78.000000,0.000000
75%,56.000000,3.000000,1.0,1.0,20.000000,0.000000,0.000000,1.000000,0.000000,264.000000,144.00000,90.000000,28.040000,83.000000,87.000000,0.000000
max,70.000000,4.000000,1.0,1.0,70.000000,1.000000,1.000000,1.000000,1.000000,696.000000,295.00000,142.500000,56.800000,143.000000,394.000000,1.000000


Taking a look into the major information of each column. The 'cigsPerDay' column shows a very high maximum value, which is questionable. Smoking 70 cigarettes a day is extremely high, but it is possible for a small subset of heavy smokers. Such cases, while rare, often correlate with addiction or chronic smoking habits developed over many years. However, in a dataset, this number could also result from data entry errors or it can be an outlier.

Another questionable value is the column's 'sysBP' maximum value. A systolic blood pressure value of 295 mmHg is exceptionally high and is considered a medical emergency. This could occur in rare and severe cases of hypertensive crisis, which might lead to organ damage if not treated immediately. Therefore, it would be strange to have a patient in the survey which needs an immediate help. This value can be a data entry error, unvalidated data, or measurement error. The same is with the 'diaBP' maximum value.

Therefore, let's check more information about those patients who have these unusual and questionable values.

In [77]:
max_cigs = chd_data[chd_data["cigsPerDay"] == 70]

max_sysbp = chd_data[chd_data["sysBP"] == 295]

max_diabp = chd_data[chd_data["diaBP"] == 142.5]


print("Row with 'cigsPerDay' = 70:")
display(max_cigs)

print("\nRow with 'sysBP' = 295:")
display(max_sysbp)

print("\nRow with 'diaBP' = 142.5:")
display(max_diabp)

Row with 'cigsPerDay' = 70:


,age,education,sex,is_smoking,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
id,,,,,,,,,,,,,,,,
2985,40,3.0,1,1,70.0,0.0,0,1,0,210.0,132.0,86.0,31.57,98.0,80.0,0



Row with 'sysBP' = 295:


,age,education,sex,is_smoking,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
id,,,,,,,,,,,,,,,,
1941,64,1.0,0,0,0.0,0.0,0,1,0,253.0,295.0,135.0,38.82,92.0,70.0,1



Row with 'diaBP' = 142.5:


,age,education,sex,is_smoking,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
id,,,,,,,,,,,,,,,,
2325,48,1.0,0,0,0.0,0.0,0,1,0,169.0,243.0,142.5,28.49,85.0,77.0,0


The additional information does not indicate an inconsistency in data collection. Therefore, these values will be retained as extreme outliers for this project and will be dealt with later.

Let's check the unique values in the integer and categorical data type columns to see if there are any unusual entries.

In [78]:
def display_unique_values(dataset: pd.DataFrame, columns: List[str]) -> None:
    """
    Display unique values for specified columns in a dataset.

    Parameters:
    - dataset (pd.DataFrame): The dataset containing the columns to inspect.
    - columns (List[str]): List of column names to check for unique values.

    Returns:
    - None: This function prints the unique values for each specified column.
    """
    for column in columns:
        unique_values = dataset[column].unique()
        print(f"Unique values in '{column}': {unique_values}")


columns_to_check = [
    "education",
    "sex",
    "is_smoking",
    "BPMeds",
    "prevalentStroke",
    "prevalentHyp",
    "TenYearCHD",
]
display_unique_values(chd_data, columns_to_check)

Unique values in 'education': [ 2.  4.  1.  3. nan]
Unique values in 'sex': <IntegerArray>
[0, 1]
Length: 2, dtype: Int64
Unique values in 'is_smoking': <IntegerArray>
[1, 0]
Length: 2, dtype: Int64
Unique values in 'BPMeds': [ 0. nan  1.]
Unique values in 'prevalentStroke': [0 1]
Unique values in 'prevalentHyp': [0 1]
Unique values in 'TenYearCHD': [1 0]


Some columns have missing values. First, let's check for all missing values across all columns, including those with float data types.

In [79]:
chd_data.isnull().sum().sort_values(ascending=False)

glucose            304
education           87
BPMeds              44
totChol             38
cigsPerDay          22
BMI                 14
heartRate            1
age                  0
prevalentHyp         0
prevalentStroke      0
sex                  0
is_smoking           0
diaBP                0
sysBP                0
diabetes             0
TenYearCHD           0
dtype: int64

However, it is important to see not only raw numbers, but also the proportion of missing values in each column.

In [80]:
(chd_data.isnull().mean().sort_values(ascending=False) * 100).round(2)

glucose            8.97
education          2.57
BPMeds             1.30
totChol            1.12
cigsPerDay         0.65
BMI                0.41
heartRate          0.03
age                0.00
prevalentHyp       0.00
prevalentStroke    0.00
sex                0.00
is_smoking         0.00
diaBP              0.00
sysBP              0.00
diabetes           0.00
TenYearCHD         0.00
dtype: float64

Out of 16 columns, 7 have missing values. The 'glucose' column has the highest number (304) and percentage (8.97%) of missing data. It is important to address these missing values, as they can affect the performance of logistic regression models.

Dealing with missing values will vary depending on the specific column. For the 'glucose', 'education', 'totChol', 'BMI', and 'heartRate' columns, missing values will be replaced with either the mean or median, based on the data variation observed in the summary statistics.

In [81]:
columns_to_fill = ["glucose", "totChol", "BMI", "heartRate"]

for column in columns_to_fill:
    median_value = chd_data[column].median()
    chd_data[column] = chd_data[column].fillna(median_value)

In the cardiovascular dataset, the values 1, 2, 3, and 4 in the 'education' column likely represent different levels of educational attainment:

1 - Primary or incomplete schooling

2 - High school

3 - Vocational or college

4 - Higher education

In [82]:
chd_data["education"] = chd_data["education"].fillna(2)  # Mean value

The 'cigsPerDay' column missing values will be replaced by 1) taking into account whether the person is smoker or not, 2) replacing the median (due to an outliers and high variation in data) value depending on gender. 

In [83]:
def fill_cigs_per_day(row: pd.Series, chd_data: pd.DataFrame) -> float:
    """
    Fills missing values in the 'cigsPerDay' column based on the following logic:
    1. If 'is_smoking' is 0, the value is filled with 0.
    2. If 'is_smoking' is not 0, the value is filled with the median value of 'cigsPerDay'
       grouped by the 'sex' column (0 for female, 1 for male).

    Args:
        row (pd.Series): A single row of the DataFrame to check and process.
        chd_data (pd.DataFrame): The DataFrame containing the full dataset, used to calculate medians based on 'sex'.

    Returns:
        float: The value to fill in 'cigsPerDay', either 0 or the median value based on 'sex'.
    """
    if row["is_smoking"] == 0:
        return 0
    else:
        median_value = chd_data[chd_data["sex"] == row["sex"]]["cigsPerDay"].median()
        return median_value


chd_data["cigsPerDay"] = chd_data.apply(
    lambda row: (
        fill_cigs_per_day(row, chd_data)
        if pd.isna(row["cigsPerDay"])
        else row["cigsPerDay"]
    ),
    axis=1,
)

The 'BPMeds' column missing values will be changed taking into account other columns: 'diaBP', 'sysBP'. The 'BPMeds' column show wether or not the patient was on blood pressure medication.

Patients may need blood pressure medication if they have high systolic ('sysBP') or diastolic ('diaBP') blood pressure. 

Patients are typically recommended to consider blood pressure medication starting from Hypertension Stage 1 (130–139 mm Hg systolic or 80–89 mm Hg diastolic) if they have additional cardiovascular risk factors, such as diabetes, heart disease, or a history of stroke. For patients with Stage 2 Hypertension (140/90 mm Hg or higher), medication is generally recommended regardless of other risk factors to reduce the risk of severe complications.

This approach aligns with guidelines from the [American Heart Association](https://www.mayoclinic.org/diseases-conditions/high-blood-pressure/in-depth/blood-pressure/art-20050982).

For this analysis, as it is only 1.3% missing values in the 'BPMeds' column, we will set boundaries for 0 (patient was NOT on blood pressure medication) if the 'sysBP' >= 140 mm Hg or 'diaBP' >= 90 mm Hg. 

In [84]:
def fill_bpmeds(row: pd.Series) -> int:
    """
    Fills missing values in the 'BPMeds' column based on the following logic:
    1. If 'sysBP' >= 140 or 'diaBP' >= 90, fill with 1.
    2. Otherwise, fill with 0.

    Args:
        row (pd.Series): A single row of the DataFrame to check and process.

    Returns:
        int: The value to fill in 'BPMeds', either 1 or 0 based on blood pressure values.
    """
    if row["sysBP"] >= 140 or row["diaBP"] >= 90:
        return 1
    else:
        return 0


chd_data["BPMeds"] = chd_data.apply(
    lambda row: fill_bpmeds(row) if pd.isna(row["BPMeds"]) else row["BPMeds"], axis=1
)

Checking if all missing values were filled:

In [85]:
(chd_data.isnull().mean().sort_values(ascending=False) * 100).round(2)

age                0.0
education          0.0
sex                0.0
is_smoking         0.0
cigsPerDay         0.0
BPMeds             0.0
prevalentStroke    0.0
prevalentHyp       0.0
diabetes           0.0
totChol            0.0
sysBP              0.0
diaBP              0.0
BMI                0.0
heartRate          0.0
glucose            0.0
TenYearCHD         0.0
dtype: float64

There are NO duplicates, **all rows are unique.**

In [86]:
uniques = chd_data.drop_duplicates()

uniques.shape

(3390, 16)

Second inspection for outliers in the numeric columns via the IQR method to see how many outliers are there. Those columns, that didn't show any potential outliers in describe table, such as 'age', 'education', will be exluded out of this inspection.

In [87]:
numeric_columns = [
    "cigsPerDay",
    "glucose",
    "heartRate",
    "BMI",
    "sysBP",
    "diaBP",
    "totChol",
]

outliers_dict = {}

for column in numeric_columns:
    Q1 = chd_data[column].quantile(0.25)
    Q3 = chd_data[column].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = chd_data[
        (chd_data[column] < lower_bound) | (chd_data[column] > upper_bound)
    ]

    outliers_dict[column] = len(outliers)

sorted_outliers = sorted(outliers_dict.items(), key=lambda x: x[1], reverse=True)

for column, num_outliers in sorted_outliers:
    print(f"Column: {column}. Number of outliers: {num_outliers}")

Column: glucose. Number of outliers: 214
Column: sysBP. Number of outliers: 105
Column: BMI. Number of outliers: 79
Column: heartRate. Number of outliers: 64
Column: diaBP. Number of outliers: 58
Column: totChol. Number of outliers: 43
Column: cigsPerDay. Number of outliers: 9


It will be dealt with outliers later on in this project

Lastly, for a data cleaning part it is convenient to have all the columns written in the same case. We will use snake_case.

In [88]:
chd_data = chd_data.rename(
    columns={
        "cigsPerDay": "cigs_per_day",
        "BPMeds": "bp_meds",
        "prevalentStroke": "prevalent_stroke",
        "prevalentHyp": "prevalent_hyp",
        "totChol": "tot_chol",
        "sysBP": "sys_bp",
        "diaBP": "dia_bp",
        "BMI": "bmi",
        "heartRate": "heart_rate",
        "TenYearCHD": "ten_year_chd",
    }
)

chd_data.head()

,age,education,sex,is_smoking,cigs_per_day,bp_meds,prevalent_stroke,prevalent_hyp,diabetes,tot_chol,sys_bp,dia_bp,bmi,heart_rate,glucose,ten_year_chd
id,,,,,,,,,,,,,,,,
0,64,2.0,0,1,3.0,0.0,0,0,0,221.0,148.0,85.0,25.38,90.0,80.0,1
1,36,4.0,1,0,0.0,0.0,0,1,0,212.0,168.0,98.0,29.77,72.0,75.0,0
2,46,1.0,0,1,10.0,0.0,0,0,0,250.0,116.0,71.0,20.35,88.0,94.0,0
3,50,1.0,1,1,20.0,0.0,0,1,0,233.0,158.0,88.0,28.26,68.0,94.0,1
4,64,1.0,0,1,30.0,0.0,0,0,0,241.0,136.5,85.0,26.42,70.0,77.0,0
